In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
#export
from exp.nb_07 import *

## ConvNet

In [ ]:
x_train,y_train,x_valid,y_valid = get_data()

x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [ ]:
mnist_view = view_tfm(1,28,28)
cbfs = [Recorder,
        partial(AvgStatsCallback,accuracy),
        CudaCallback,
        partial(BatchTransformXCallback, mnist_view)]

In [ ]:
nfs = [8,16,32,64,64]

In [ ]:
def conv_layer(ni, nf, ks=3, stride=2, **kwargs):
    return nn.Sequential(
        nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride, bias=True),
        GeneralRelu(**kwargs))

In [ ]:
learn,run = get_learn_run(nfs, data, 0.4, conv_layer, cbs=cbfs)

In [ ]:
def get_batch(dl, run):
    run.xb,run.yb = next(iter(dl))
    for cb in run.cbs: cb.set_runner(run)
    run('begin_batch')
    return run.xb,run.yb

In [ ]:
xb,yb = get_batch(data.train_dl, run)

In [ ]:
def find_modules(m, cond):
    if cond(m): return [m]
    return sum([find_modules(o,cond) for o in m.children()], [])

In [ ]:
def is_lin_layer(l):
    lin_layers = (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear)
    return isinstance(l, lin_layers)

In [ ]:
mods = find_modules(learn.model, is_lin_layer)

In [ ]:
def append_mean(hook, mod, inp, outp):
    d = outp.data
    hook.mean,hook.std = d.mean().item(),d.std().item()

In [ ]:
mdl = learn.model.cuda()

In [ ]:
with Hooks(mods,append_mean) as hooks:
    mdl(xb)
    for hook in hooks: print(hook.mean,hook.std)

0.013466657139360905 1.251402735710144
-0.06908904761075974 1.2406070232391357
-0.08122466504573822 1.0131332874298096
0.063581183552742 0.8145366311073303
-0.006549822632223368 0.5609457492828369
-0.03177064284682274 0.2738892436027527


In [ ]:
def lsuv_module(m, xb):
    h = Hook(m, append_mean)

    if getattr(m, 'bias', None) is not None:
        while mdl(xb) is not None and abs(h.mean) > 1e-3:
            m.bias.data -= h.mean

    while mdl(xb) is not None and abs(h.std-1) > 1e-3:
        m.weight.data /= h.std

    h.remove()
    return h.mean,h.std

In [ ]:
for m in mods: print(lsuv_module(m, xb))

(-0.002705414080992341, 1.0)
(-0.0009297980577684939, 0.9999998807907104)
(-0.013155314140021801, 1.0000001192092896)
(0.01316459197551012, 0.9999998807907104)
(-0.005277046002447605, 1.0)
(-0.0597885362803936, 0.9991505742073059)


In [ ]:
%time run.fit(2, learn)

train: [0.995904375, tensor(0.6753, device='cuda:0')]
valid: [0.20251124267578124, tensor(0.9405, device='cuda:0')]
train: [0.158788125, tensor(0.9516, device='cuda:0')]
valid: [0.1106400634765625, tensor(0.9672, device='cuda:0')]
CPU times: user 2.22 s, sys: 448 ms, total: 2.67 s
Wall time: 2.67 s
